## Code for LLaMa-Apter Tuning

Find the code for tuning the LLaMa-7B mode using the adapter method below. It is an implementation of LLaMA-tuninig which is "fully open source under the Apache 2.0 license."

In [ ]:
!pip install huggingface
!pip install huggingface_hub


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Your token has been saved in your 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

https://github.com/Lightning-AI/lit-llama/blob/main/howto/finetune_adapter.md

In [ ]:
!git clone https://github.com/Lightning-AI/lit-llama ## this first ?!

fatal: destination path 'lit-llama' already exists and is not an empty directory.


In [ ]:
!git clone https://huggingface.co/meta-llama/Llama-2-7b-hf checkpoints/open-llama/7B



fatal: destination path 'checkpoints/open-llama/7B' already exists and is not an empty directory.


In [ ]:
!pip install -r lit-llama/requirements.txt
# might take some time dpeneding on internet connectivity


  Cloning https://github.com/Lightning-AI/lightning (to revision master) to /tmp/pip-install-ehnj6kvh/lightning_236002036eb0416897e19490233a750a
  Running command git clone --filter=blob:none --quiet https://github.com/Lightning-AI/lightning /tmp/pip-install-ehnj6kvh/lightning_236002036eb0416897e19490233a750a
  Resolved https://github.com/Lightning-AI/lightning to commit dab373de548b352a69c8ea7251937fb4663038dc
  Running command git submodule update --init --recursive -q
  Encountered 31 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy

In [ ]:
!python lit-llama/scripts/convert_hf_checkpoint.py --checkpoint_dir checkpoints/open-llama/7B --model_size 7B

Initializing lit-llama
Saving to disk at checkpoints/lit-llama/7B
Processing checkpoints/open-llama/7B/pytorch_model-00002-of-00003.bin
Processing checkpoints/open-llama/7B/pytorch_model-00003-of-00003.bin
Processing checkpoints/open-llama/7B/pytorch_model-00001-of-00003.bin


## Finetuning try 1

https://github.com/Lightning-AI/lit-llama/blob/main/howto/finetune_adapter.md

In [ ]:
# connecting to drive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


### Loading in data from drive and setting up own dataloader (.pt) files like the function in the python file prepare_alpacy do.

In [ ]:

import pandas as pd
airbnb_london_filtered_images = pd.read_csv("/content/gdrive/My Drive/Thesis/London_Data/discriminator_tabular_data_images.csv")

In [ ]:
import numpy as np
frac_train_size = 0.7

## now using all samples!

train_eval_airbnb_london_filtered_images = airbnb_london_filtered_images

train_airbnb_london_filtered_advanced = train_eval_airbnb_london_filtered_images.sample(n = int(np.ceil(frac_train_size*train_eval_airbnb_london_filtered_images.shape[0])), random_state = 100)
eval_airbnb_london_filtered_advanced = train_eval_airbnb_london_filtered_images.drop(train_airbnb_london_filtered_advanced.index, axis = 0)

train_airbnb_london_filtered_advanced.index = list(range(train_airbnb_london_filtered_advanced.shape[0]))
eval_airbnb_london_filtered_advanced.index = list(range(eval_airbnb_london_filtered_advanced.shape[0]))


Logical dependencies of functions in prepare_alpacy file

'prepare' uses
> 'prepare_sample' uses
>> 'tokenzize' & 'generate promt'



In [ ]:
## Using the following instruction:

instruction = "Summarize the following description into a short title for an AirBnB listing."


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.3 MB/s eta 0:00:00


In [ ]:
import os
from pathlib import Path
from typing import Optional
import torch
from sentencepiece import SentencePieceProcessor, SentencePieceTrainer
import sys
import torch
import requests
import json
from torch.utils.data import random_split
from tqdm import tqdm

class Tokenizer:
    """Tokenizer for LLaMA."""

    def __init__(self, model_path: Path) -> None:
        self.processor = SentencePieceProcessor(model_file=str(model_path))
        self.bos_id = self.processor.bos_id()
        self.eos_id = self.processor.eos_id()
        self.pad_id = self.processor.pad_id()

    @property
    def vocab_size(self) -> int:
        return self.processor.vocab_size()

    def encode(
        self,
        string: str,
        bos: bool = True,
        eos: bool = False,
        max_length: int = -1,
        pad: bool = False,
        device: Optional[torch.device] = None
    ) -> torch.Tensor:
        tokens = self.processor.encode(string)
        if bos:
            tokens = [self.bos_id] + tokens
        if eos:
            tokens = tokens + [self.eos_id]
        if max_length > 0:
            tokens = tokens[:max_length]
        if pad and len(tokens) < max_length:
            tokens += [self.pad_id] * (max_length - len(tokens))

        return torch.tensor(tokens, dtype=torch.int, device=device)

    def decode(self, tokens: torch.Tensor) -> str:
        return self.processor.decode(tokens.tolist())

    @staticmethod
    def train(input: str, destination: str, vocab_size=32000) -> None:
        model_prefix = os.path.join(destination, "tokenizer")
        SentencePieceTrainer.Train(input=input, model_prefix=model_prefix, vocab_size=vocab_size)


## general function
def tokenize(tokenizer: Tokenizer, string: str, max_length: int, eos=True) -> torch.Tensor:
    return tokenizer.encode(string, bos=True, eos=eos, max_length=max_length)

## general function

def generate_prompt(example):
    """Generates a standardized message to prompt the model with an instruction, optional input and a
    'response' field."""

    if example["input"]:
        return (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:"
        )
    return (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{example['instruction']}\n\n### Response:"
    )


def prepare_sample(example: dict, tokenizer: Tokenizer, max_length: int, mask_inputs: bool = True):
    """Processes a single sample.

    Each sample in the dataset consists of:
    - instruction: A string describing the task
    - input: A string holding a special input value for the instruction.
        This only applies to some samples, and in others this is empty.
    - output: The response string

    This function processes this data to produce a prompt text and a label for
    supervised training. The input text is formed as a single message including all
    the instruction, the input (optional) and the response.
    The label/target is the same message but can optionally have the instruction + input text
    masked out (mask_inputs=True).

    Finally, both the prompt and the label get tokenized. If desired, all tokens
    in the label that correspond to the original input prompt get masked out (default).
    """

    full_prompt = generate_prompt(example)
    full_prompt_and_response = full_prompt + example["output"]
    encoded_full_prompt = tokenize(tokenizer, full_prompt, max_length=max_length, eos=False)
    encoded_full_prompt_and_response = tokenize(tokenizer, full_prompt_and_response, eos=True, max_length=max_length)

    # The labels are the full prompt with response, but with the prompt masked out
    labels = encoded_full_prompt_and_response.clone()
    if mask_inputs:
        labels[:len(encoded_full_prompt)]  = -1 ## corresponds to "ignore index" inprepare_alpaca.py file


    return {**example, "input_ids": encoded_full_prompt_and_response, "input_ids_no_response": encoded_full_prompt, "labels": labels}




#### prepare function is not required!

In [ ]:
tokenizer = Tokenizer("/content/checkpoints/lit-llama/tokenizer.model")

Reocding the prepare function:

In [ ]:
## setting up new dict

data_dict_train  = []
data_dict_eval  = []

In [ ]:

for i in range(train_airbnb_london_filtered_advanced.shape[0]):

  data_dict_train.append({
      "instruction": "Summarize the following description into a short title for an AirBnB listing.",
      "input":   train_airbnb_london_filtered_advanced.description[i],
      "output": train_airbnb_london_filtered_advanced.name[i]
  })

for i in range(eval_airbnb_london_filtered_advanced.shape[0]):

  data_dict_eval.append({
      "instruction": "Summarize the following description into a short title for an AirBnB listing.",
      "input":   eval_airbnb_london_filtered_advanced.description[i],
      "output": eval_airbnb_london_filtered_advanced.name[i]
  })


Create a new folder called "data"

In [ ]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
print("Processing train split ...")
max_seq_length = 256
mask_inputs = True
train_set = [prepare_sample(sample, tokenizer, max_seq_length, mask_inputs) for sample in data_dict_train]
torch.save(train_set, "/content/data/train.pt")




Processing train split ...


In [ ]:
print("Processing test split ...")
max_seq_length = 256
mask_inputs = True
test_set = [prepare_sample(sample, tokenizer, max_seq_length, mask_inputs) for sample in data_dict_eval]
torch.save(train_set, "/content/data/test.pt")


Processing test split ...


#### Actual fine-tuning

The finetuning file is called. However, a few changes need to be made to this file (adapter.py)

* in line 46, the size must be set to 4048 (size of my training set)
* in line 61, 'data_dict' must be set to "data"
* in line 63, 'out_dir' can be set to "out/adapter/airbnb" (Not necessary, just nicer!)

In [ ]:
## ggf example dict am anfang des daataloaders noch neu coden!


In [ ]:
## in adapter py file


# line 46
epoch_size = len(train_set)
epoch_size

# 61 data_dir -> "data/"
# 63 out_dir = out/adapter/airbnb ## not requiredd, just nicer!



15799

In [ ]:
!python lit-llama/finetune/adapter.py

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
iter 14758: loss 2.1499, time: 169.17ms
iter 14759: loss 1.3615, time: 169.67ms
iter 14760: loss 1.9305, time: 169.71ms
iter 14761: loss 1.5204, time: 169.59ms
iter 14762: loss 4.4675, time: 169.40ms
iter 14763: loss 1.6808, time: 169.81ms
iter 14764: loss 1.6820, time: 169.20ms
iter 14765: loss 0.7466, time: 169.38ms
iter 14766: loss 1.3951, time: 169.64ms
iter 14767: loss 1.4467, time: 171.92ms
iter 14768: loss 1.8322, time: 168.22ms
iter 14769: loss 1.7936, time: 169.45ms
iter 14770: loss nan, time: 169.56ms
iter 14771: loss 1.8184, time: 169.20ms
iter 14772: loss 1.8302, time: 169.29ms
iter 14773: loss 1.7570, time: 169.69ms
iter 14774: loss 1.7404, time: 169.93ms
iter 14775: loss 2.0740, time: 169.26ms
iter 14776: loss 1.4096, time: 169.45ms
iter 14777: loss 2.5912, time: 169.27ms
iter 14778: loss 2.2126, time: 169.26ms
iter 14779: loss 1.4681, time: 169.02ms
iter 14780: loss 1.8974, time: 169.67ms
iter 14781: loss

In [ ]:

# all in all ca 180 min training time

In [ ]:
## Savign model by downloading and uploading to drive

## Later examining generate function!


In [ ]:
!mkdir out

mkdir: cannot create directory ‘out’: File exists


In [ ]:
mode_ft = torch.load("/content/gdrive/My Drive/Thesis/Models/lit-llama-adapter-finetuned_all.pth")
torch.save(mode_ft, "/content/out/lit-llama-adapter-finetuned.pth")


In [ ]:
sys.path.append("/content/lit-llama")


import lightning as L
import torch
from generate import generate
from lit_llama import Tokenizer
from lit_llama.adapter import LLaMA
from lit_llama.utils import EmptyInitOnDevice, lazy_load, llama_model_lookup
from scripts.prepare_alpaca import generate_prompt

In [ ]:
adapter_path ="/content/out/lit-llama-adapter-finetuned.pth"
pretrained_path = "/content/checkpoints/lit-llama/7B/lit-llama.pth"
tokenizer_path = "/content/checkpoints/lit-llama/tokenizer.model"


fabric = L.Fabric(devices=1)
dtype = torch.bfloat16 if fabric.device.type == "cuda" and torch.cuda.is_bf16_supported() else torch.float32


The following function runs slowly, but sadly is needed to call for every generation ....

In [ ]:
def gen_title(pretrained_path, adapter_path, sample, max_new_tokens, top_k, temperature):

  """ Expects sample as set up above / like in the .pt data """

  with lazy_load(pretrained_path) as pretrained_checkpoint, lazy_load(adapter_path) as adapter_checkpoint:
      name = llama_model_lookup(pretrained_checkpoint)

      with EmptyInitOnDevice(
              device=fabric.device, dtype=dtype, quantization_mode = "llm.int8"
      ):
      #   quantization
          model = LLaMA.from_name(name)


      # 1. Load the pretrained weights
      model.load_state_dict(pretrained_checkpoint, strict=False)
      # 2. Load the fine-tuned adapter weights
      model.load_state_dict(adapter_checkpoint, strict=False)


  model.eval()
  model = fabric.setup_module(model)

  tokenizer = Tokenizer(tokenizer_path)

  prompt = generate_prompt(sample)
  encoded = tokenizer.encode(prompt, bos=True, eos=False, device=model.device)
  prompt_length = encoded.size(0)

  y = generate(model, encoded, max_new_tokens, temperature=temperature, top_k=top_k, eos_id=tokenizer.eos_id)

  output = tokenizer.decode(y)
  output = output.split("### Response:")[1].strip()

  return output


Finally, I shall generate titles. As this is time intensive, I shall only generate 1000 samples

In [ ]:
sub_df = pd.read_csv("/content/gdrive/My Drive/Thesis/loss_data/gen_titles_llama.csv")
sub_df_ids = list(sub_df.id)

In [ ]:
airbnb_london_filtered_images_usable = airbnb_london_filtered_images[airbnb_london_filtered_images.id.isin(sub_df_ids)]

In [ ]:
airbnb_london_filtered_images_usable.index = list(range(airbnb_london_filtered_images_usable.shape[0]))

In [ ]:
## setting up new dict

data_dict_sub_df = []

for i in range(airbnb_london_filtered_images_usable.shape[0]):

  data_dict_sub_df.append({

      "instruction": "Summarize the following description into a short, appealing title for an AirBnB listing.",
      "input":   airbnb_london_filtered_images_usable.description[i],
      "output": airbnb_london_filtered_images_usable.name[i]
  })


max_seq_length = 256
mask_inputs = True
gen_set = [prepare_sample(sample, tokenizer, max_seq_length, mask_inputs) for sample in data_dict_sub_df]


In [ ]:
list_ids_ = list(airbnb_london_filtered_images_usable.id)

In [ ]:
gen_titles = []
list_ids = []

for i in range(len(gen_set)):
  sample_ = gen_set[i]
  gen_title_new = gen_title(pretrained_path, adapter_path, sample_, 30, 200, 0.2)
  gen_titles.append(gen_title_new)
  list_ids.append(list_ids_[i])

  if i % 100 == 0:
    sub_df_adapter = pd.DataFrame({"id": list_ids, "gen_titles":gen_titles})
    sub_df_adapter.to_csv("/content/gdrive/My Drive/Thesis/loss_data/gen_titles_llama_adapter_v2_all.csv", index = False)

sub_df_adapter = pd.DataFrame({"id": list_ids, "gen_titles":gen_titles})
sub_df_adapter.to_csv("/content/gdrive/My Drive/Thesis/loss_data/gen_titles_llama_adapter_v2_all.csv", index = False)


In [ ]:
from google.colab import runtime
runtime.unassign()